# WhisperX Cloud API Server 🚀 (Universal)

**支持平台:** Google Colab | Kaggle | 本地 GPU 服务器

这个 Notebook 自动检测运行环境并部署 WhisperX API 服务。

## 🎯 特性
- ✅ 自动检测环境 (Colab/Kaggle/Local)
- ✅ 自动 GPU/CPU 检测
- ✅ 内置 ngrok 隧道（公共 URL）
- ✅ 支持多语言
- ✅ 单词级时间戳对齐
- ✅ 可选说话人分离

## 📋 使用步骤
1. **设置 GPU**: 在 Colab/Kaggle 中启用 GPU
2. **配置 ngrok**: 在下面设置 ngrok token
3. **运行所有单元格**: 点击 Runtime → Run all
4. **复制 URL**: 将生成的公共 URL 复制到 VideoLingo 配置

---

## 0️⃣ 准备项目文件

如果是 Colab/Kaggle 环境，需要先克隆项目文件。本地运行可跳过此步骤。

In [ ]:
import os
import sys

# 检测环境
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = os.path.exists('/kaggle')

# 设置持久化目录
if IN_COLAB:
    # Colab: 使用 Google Drive 或 /content (重启后保留)
    PERSIST_DIR = '/content/videolingo-whisperx'
elif IN_KAGGLE:
    # Kaggle: 使用 /kaggle/working
    PERSIST_DIR = '/kaggle/working/videolingo-whisperx'
else:
    PERSIST_DIR = '.'

REPO_URL = 'https://github.com/infinite-gaming-studio/VideoLingo.git'

if IN_COLAB or IN_KAGGLE:
    # 创建持久化目录
    os.makedirs(PERSIST_DIR, exist_ok=True)
    
    # 检查是否已有仓库
    repo_dir = os.path.join(PERSIST_DIR, 'VideoLingo')
    
    if os.path.exists(os.path.join(repo_dir, '.git')):
        print("🔄 更新项目代码...")
        !cd {repo_dir} && git pull
    else:
        print("📥 克隆项目到持久化目录...")
        !git clone {REPO_URL} {repo_dir}
    
    # 复制文件到当前工作目录
    !cp -r {repo_dir}/whisperx_cloud/* .
    print(f"✅ 项目文件已同步到: {PERSIST_DIR}")
else:
    print("✅ 本地环境: 使用当前目录文件")

## 1️⃣ 环境配置

In [ ]:
%run config.py

## 2️⃣ 环境检测与设置

In [ ]:
%run step1_detect_env.py

## 3️⃣ GPU 检查

In [ ]:
%run step2_check_gpu.py

## 4️⃣ 安装 Conda 和依赖

本步骤会：
1. 检查并安装 Miniconda（如果未安装）
2. 使用 Conda 创建环境并安装依赖

### 依赖版本说明 (参考 VideoLingo 父项目):
- `torch==2.0.0` - 与 VideoLingo 保持一致
- `whisperx@git+...853` - 固定 commit 保证稳定性
- `ctranslate2==4.4.0` - whisperX 依赖的推理引擎
- `transformers==4.39.3` - HuggingFace 模型库

In [ ]:
import os
import sys
import subprocess

# ============================================
# Step 4.1: 安装 Conda (如果需要)
# ============================================

CONDA_CMD = None

# 检查 Conda 是否已安装
try:
    result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        CONDA_CMD = 'conda'
        print(f"✅ Conda detected: {result.stdout.strip()}")
except:
    pass

# 检查用户目录 miniconda
if not CONDA_CMD:
    miniconda_conda = os.path.expanduser('~/miniconda3/bin/conda')
    if os.path.exists(miniconda_conda):
        CONDA_CMD = miniconda_conda
        os.environ['PATH'] = os.path.expanduser('~/miniconda3/bin:') + os.environ.get('PATH', '')
        result = subprocess.run([CONDA_CMD, '--version'], capture_output=True, text=True)
        print(f"✅ Miniconda detected: {result.stdout.strip()}")

# 安装 Miniconda
if not CONDA_CMD:
    print("📥 Installing Miniconda...")
    !wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
    !bash /tmp/miniconda.sh -b -p ~/miniconda3
    CONDA_CMD = os.path.expanduser('~/miniconda3/bin/conda')
    os.environ['PATH'] = os.path.expanduser('~/miniconda3/bin:') + os.environ.get('PATH', '')
    print("✅ Miniconda installed")

# ============================================
# Step 4.2: 安装依赖
# ============================================

print("\n📦 Installing dependencies with Conda...\n")

# 检测平台
IN_KAGGLE = os.path.exists('/kaggle')

# Kaggle 持久化目录设置
if IN_KAGGLE:
    ENV_PREFIX = '/kaggle/working/conda-envs/whisperx-cloud'
    os.makedirs('/kaggle/working/conda-envs', exist_ok=True)
    os.environ['HF_HOME'] = '/kaggle/working/.cache/huggingface'
    os.environ['TORCH_HOME'] = '/kaggle/working/.cache/torch'
    os.environ['CONDA_PKGS_DIRS'] = '/kaggle/working/.cache/conda/pkgs'
    for d in [os.environ['HF_HOME'], os.environ['TORCH_HOME'], os.environ['CONDA_PKGS_DIRS']]:
        os.makedirs(d, exist_ok=True)
    print("📂 Kaggle: Using persistent directory")
else:
    ENV_PREFIX = None
    print("📂 Local: Using default conda env location")

# 创建环境文件
environment_yml = '''name: whisperx-cloud
channels:
  - pytorch
  - nvidia
  - conda-forge
  - defaults
dependencies:
  - python=3.10
  - pytorch=2.0.0
  - torchaudio=2.0.0
  - pytorch-cuda=11.8
  - pip
  - pip:
    - fastapi==0.109.0
    - uvicorn[standard]==0.27.0
    - python-multipart==0.0.6
    - pydantic==2.5.3
    - requests
    - pyngrok
    - whisperx@git+https://github.com/m-bain/whisperx.git@7307306a9d8dd0d261e588cc933322454f853853
'''

with open('environment.yml', 'w') as f:
    f.write(environment_yml)
print("📝 Created environment.yml")

# 检查环境是否已存在
if ENV_PREFIX:
    env_exists = os.path.exists(ENV_PREFIX)
else:
    result = subprocess.run([CONDA_CMD, 'env', 'list'], capture_output=True, text=True)
    env_exists = 'whisperx-cloud' in result.stdout

# 创建或更新环境
if env_exists:
    print("\n🔄 Environment exists, updating...")
    if ENV_PREFIX:
        !{CONDA_CMD} env update -f environment.yml --prefix {ENV_PREFIX} --yes
    else:
        !{CONDA_CMD} env update -f environment.yml -n whisperx-cloud --yes
else:
    print("\n🆕 Creating new environment...")
    if ENV_PREFIX:
        !{CONDA_CMD} env create -f environment.yml --prefix {ENV_PREFIX} --yes
    else:
        !{CONDA_CMD} env create -f environment.yml --yes

print("\n✅ Conda environment setup complete!")

# 获取 conda 环境的 Python 路径
if ENV_PREFIX:
    CONDA_PYTHON = f"{ENV_PREFIX}/bin/python"
else:
    CONDA_PYTHON = os.path.expanduser('~/miniconda3/envs/whisperx-cloud/bin/python')

# 保存配置供后续步骤使用
with open('.conda_python_path', 'w') as f:
    f.write(CONDA_PYTHON)

# 在 conda 环境中安装 pyngrok
print("\n📦 Installing pyngrok in conda environment...")
!{CONDA_PYTHON} -m pip install -q pyngrok
print("✅ pyngrok installed")

# 保存配置供后续步骤使用
with open('.conda_python_path', 'w') as f:
    f.write(CONDA_PYTHON)

print(f"\n📌 Conda Python path saved: {CONDA_PYTHON}")
print("\n⚠️  IMPORTANT: 后续步骤将自动使用 conda 环境")

if IN_KAGGLE:
    print(f"\n📌 KAGGLE: Environment at {ENV_PREFIX}")

## 5️⃣ 检查 API 服务器

API 服务器代码位于 `whisperx_server.py`，已预先配置好。

In [ ]:
import os
if os.path.exists('whisperx_server.py'):
    print('✅ Server file found: whisperx_server.py')
else:
    print('❌ Server file not found!')

## 6️⃣ 配置 ngrok

自动使用 conda 环境的 Python

In [ ]:
import os

# 读取 conda Python 路径
try:
    with open('.conda_python_path', 'r') as f:
        CONDA_PYTHON = f.read().strip()
except:
    CONDA_PYTHON = 'python'

# 使用 conda 环境运行
!{CONDA_PYTHON} step5_setup_ngrok.py

## 7️⃣ 启动服务器

自动使用 conda 环境的 Python

In [ ]:
# 使用 conda 环境运行
!{CONDA_PYTHON} step6_start_server.py

## 8️⃣ 测试 API

自动使用 conda 环境的 Python

In [ ]:
# 使用 conda 环境运行
!{CONDA_PYTHON} step7_test_api.py

## 9️⃣ 保持运行 (⚠️ 不要停止此单元格!)

自动使用 conda 环境的 Python

In [ ]:
# 使用 conda 环境运行
!{CONDA_PYTHON} step8_keep_running.py

## 📚 VideoLingo 配置指南

服务器运行后，将 URL 配置到 VideoLingo:

### 方法 1: 修改 config.yaml
```yaml
whisper:
  runtime: 'cloud'
  whisperX_cloud_url: 'https://xxxx.ngrok-free.app'  # 你的 ngrok URL
```

### 方法 2: 环境变量
```bash
export WHISPERX_CLOUD_URL='https://xxxx.ngrok-free.app'
```

### 方法 3: VideoLingo 客户端
使用 `whisperx_cloud_client.py` 中的 `WhisperXCloudClient` 类

## 🔧 故障排除

### 1. ngrok 连接失败
- 检查 token 是否正确
- 确认网络连接（Kaggle 需要 Internet ON）
- 尝试重新运行第 5、6 单元格

### 2. GPU 未检测到
- Colab: Runtime → Change runtime type → GPU
- Kaggle: Settings → Accelerator → GPU T4 x2

### 3. 模型下载慢
- 设置 HF_ENDPOINT 为镜像站（中国大陆）
- 或等待首次下载完成（约 3GB）

### 4. 显存不足
- 使用较小模型: 'medium' 或 'small'
- 减小 batch_size
- 禁用 speaker_diarization